# HiMAP tutorial: human data

This tutorial assumes you've installed HiMAP.

To test, download files from 2 samples of DIABIMMUNE Tri-county cohort:
* [Sample G3225 (forward)](https://pubs.broadinstitute.org/diabimmune/data/10/G63225_R1_001.fastq.gz)
* [Sample G3225 (reverse)](https://pubs.broadinstitute.org/diabimmune/data/10/G63225_R2_001.fastq.gz)
* [Sample G3228 (forward)](https://pubs.broadinstitute.org/diabimmune/data/10/G63228_R1_001.fastq.gz)
* [Sample G3228 (reverse)](https://pubs.broadinstitute.org/diabimmune/data/10/G63228_R2_001.fastq.gz)

and save them to a folder. For the purposes of this tutorial, we will save them to `~/data/diabimmune/fastq_tutorial/`. In these samples, V4 hyper-variable region is sequenced.

In [ ]:
library(himap)

## Loading files and specifying folders

First, we will need full paths to the FASTQ files for pair-end 16S reads. Filenames for all forwards reads will be stored in `fq_fwd` and reverse in `fq_rev`.

In [ ]:
fastq_path = path.expand('~/data/diabimmune/tutorial/fastq')
fq_fwd = read_files(fastq_path, 'R1')
fq_rev = read_files(fastq_path, 'R2')

The filenames are used to extract identifiers for each file, either with forward or reverse reads. The function `sampleids_from_filenames` will retrieve those, given a separator (default is the underscore _). For example, from forward files: 

In [ ]:
head(fq_fwd)

we would like `G63225` and `G63228` labels:

In [ ]:
sample_ids = sampleids_from_filenames(fq_fwd, separator='_')
head(sample_ids)

HiMAP will output a number of output files and folders, which will be explained as we go along, so specify a folder where to save all files:

In [ ]:
out_path = path.expand('~/data/diabimmune/tutorial')

## Merging reads

First specify the output files for each sample, the merge:

In [ ]:
fq_mer = file.path(out_path, 'merged', paste0(sample_ids, '.fastq'))
mergestats = merge_pairs(fq_fwd, fq_rev, fq_mer, verbose=T)

Check how many reads are merged and not merged:

In [ ]:
colnames(mergestats) = sample_ids
mergestats

Out of 107,153 total reads 1,264 were not merged due to low percentage similarity in the alignment (if this number is high, consider lowering `min_sim` parameter) or too high alignment length (if this number is too high, consider lowering `low_aln_len`). See `?merge_pairs` for more details.

## Trim PCR primers

Trim PCR primers from merged reads. Specify region 'V4' or 'V3-V4'. First, create output filenames, then do the trimming, using V4 region primers. Exact primer sequences can also be specified using arguments `pr_fwd` and `pr_rev`; see `?pcr_primer_trimmer` for details.

In [ ]:
fq_tri = file.path(out_path, 'trimmed', paste0(sample_ids, '.fastq'))
trimstats = pcr_primer_trimmer(fq_mer, fq_tri, region='V4')

In [ ]:
colnames(trimstats) = sample_ids
trimstats

No primers were found in this dataset.

## Quality control and fixed-length trimming

Since the input for DADA2 denoising requires all sequences to be trimmed to the fixed length, we first inspect the distribution of sequence lengths before choosing this parameter:

In [ ]:
seqlen.ft = sequence_length_table(fq_tri)

In [ ]:
options(repr.plot.width=4, repr.plot.height=3) # Make a smaller plot
plot(seqlen.ft)

We can select the minimum trimming length by finding the length above which we have 99% of the reads, since most of the reads here have 252 and 253 nt length.

In [ ]:
trim_length = ftquantile(seqlen.ft, 0.01)
trim_length

Now we can use that to do the trimming. For quality control keep sequences with 2 or less expected errors [Edgar, Flyberg]. This also removes any phiX sequences and sequences containing any Ns after truncation (trimming):

In [ ]:
fq_fil = file.path(out_path, 'filtered', paste0(sample_ids, '.fastq'))
filtstats = dada2::filterAndTrim(fq_tri, fq_fil, truncLen=trim_length, maxEE=2, multithread=T, verbose=T)

As before, the function returns the table showing the number of reads kept during the QC filtering process:

In [ ]:
filtstats

## Denoising

For denoising, we use `dada` function from the dada2 package. After the denoising part is done, for each partition, we retrieve pre-trimmed sequences and add their consensus sequence back. This can improve the accuracy of the alignment to the reference database in one of the next steps.

In [ ]:
# dada_result = dada_denoise(fq_fil, fq_tri)
dada_result = readRDS(file.path(out_path, 'dada_result'))

In [ ]:
# saveRDS(dada_result, file.path(out_path, 'dada_result'))

## Count sequences, remove bimeras, collapse sequences from multiple samples

Next, we count the number of reads for each denoised sequence, remove chimeric (here bimeric) reads. If we work with multiple samples, we pool together denoised sequences that are identical up to shifts or length variation:

In [ ]:
ab.dt = sequence_abundance(dada_result)

In [ ]:
head(ab.dt, 10)

## Align sequences vs reference database

Now that we have a final list of sequences, we align these sequences to the reference database. We BLAST these sequences against our V4 region HiMAP database:

In [ ]:
blast_output_file = file.path(out_path, 'blast_vs_db.txt')
blast_output = blast(ab.dt, blast_output_file, region='V4', max_target_seqs=100, verbose=T)

## Calculate OSU abundances

We then use sequence abundance table `ab.dt` and `blast_output` to infer OSUs: 

In [ ]:
osu_ab.dt = abundance(ab.dt, blast_output)

In [ ]:
head(osu_ab.dt, 10)

In [ ]:
write_table(osu_ab.dt, file.path(out_path, 'osu_abundances.txt'))

## Add taxonomy

For each OSU, we get taxonomic ranks for all detected strains, from NCBI Taxonomy database.

In [ ]:
osu_tax.dt = taxonomy(osu_ab.dt)

In [ ]:
head(osu_tax.dt, 10)